In [1]:
import pandas as pd

In [2]:
url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13'
krx = pd.read_html(url, header=0)[0]

# add conditions 
# krx_tmp = krx_tmp[~krx_tmp['업종'].str.contains('건설')]
# krx_tmp.reset_index(drop = True, inplace = True)

krx = krx[['종목코드', '회사명']]
krx = krx.rename(columns={'종목코드': 'code', '회사명': 'company'})
# krx.rename(columns={'종목코드': 'code', '회사명': 'company'}, inplace = True)
krx.code = krx.code.map('{:06d}'.format)
# krx['code']=krx['code'].map('{:06d}'.format)

In [31]:
import pymysql

connection = pymysql.connect(host='localhost',
                             port=3307,
                             db='INVESTAR', 
                             user='root',
                             passwd='!@#$!#@#$',
                             autocommit=True) 

In [33]:
sql = "SELECT * FROM company_info"
df = pd.read_sql(sql, connection)
df.head()

,code,company,last_update
0,000020,동화약품,2020-10-26
1,000040,KR모터스,2020-10-26
2,000050,경방,2020-10-26
3,000060,메리츠화재,2020-10-26
4,000070,삼양홀딩스,2020-10-26


In [36]:
from bs4 import BeautifulSoup
import urllib, pymysql, calendar, time, json
from urllib.request import urlopen
from datetime import datetime
from threading import Timer

In [38]:
def read_krx_code(self):
        """KRX로부터 상장기업 목록 파일을 읽어와서 데이터프레임으로 반환"""
        url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method='\
            'download&searchType=13'
        krx = pd.read_html(url, header=0)[0]
        krx = krx[['종목코드', '회사명']]
        krx = krx.rename(columns={'종목코드': 'code', '회사명': 'company'})
        krx.code = krx.code.map('{:06d}'.format)
        return krx

In [46]:
codes = dict()
# curs = connection.cursor() 
with connection.cursor() as curs:
    sql = "SELECT max(last_update) FROM company_info"
    curs.execute(sql)
    rs = curs.fetchone()
    today = datetime.today().strftime('%Y-%m-%d')
    if rs[0] == None or rs[0].strftime('%Y-%m-%d') < today:
        # krx = read_krx_code()
        for idx in range(len(krx)):
            code = krx.code.values[idx]
            company = krx.company.values[idx]                
            sql = f"REPLACE INTO company_info (code, company, last"\
                f"_update) VALUES ('{code}', '{company}', '{today}')"
            curs.execute(sql)
            # key : code, value : company
            codes[code] = company
            tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
            print(f"[{tmnow}] #{idx+1:04d} REPLACE INTO company_info "\
                f"VALUES ({code}, {company}, {today})")
        connection.commit()
        print('')     

## 5.3.7

In [ ]:
def read_naver(self, code, company, pages_to_fetch):
    """네이버에서 주식 시세를 읽어서 데이터프레임으로 반환"""
    try:
        url = f"http://finance.naver.com/item/sise_day.nhn?code={code}"
        with urlopen(url) as doc:
            if doc is None:
                return None
            html = BeautifulSoup(doc, "lxml")
            pgrr = html.find("td", class_="pgRR")
            if pgrr is None:
                return None
            s = str(pgrr.a["href"]).split('=')
            lastpage = s[-1] 
        df = pd.DataFrame()
        pages = min(int(lastpage), pages_to_fetch)
        for page in range(1, pages + 1):
            pg_url = '{}&page={}'.format(url, page)
            df = df.append(pd.read_html(pg_url, header=0)[0])
            tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
            print('[{}] {} ({}) : {:04d}/{:04d} pages are downloading...'.
                format(tmnow, company, code, page, pages), end="\r")
        df = df.rename(columns={'날짜':'date','종가':'close','전일비':'diff'
            ,'시가':'open','고가':'high','저가':'low','거래량':'volume'})
        df['date'] = df['date'].replace('.', '-')
        df = df.dropna()
        df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close',
            'diff', 'open', 'high', 'low', 'volume']].astype(int)
        df = df[['date', 'open', 'high', 'low', 'close', 'diff', 'volume']]
    except Exception as e:
        print('Exception occured :', str(e))
        return None
        
    return df

In [67]:
url = f"http://finance.naver.com/item/sise_day.nhn?code=000020"
with urlopen(url) as doc:
    
    html = BeautifulSoup(doc, "lxml")
    pgrr = html.find("td", class_="pgRR")
    s = str(pgrr.a["href"]).split('=')
    lastpage = s[-1] 
    

In [86]:
code = '000020'
company = '동화약품'
pages_to_fetch = 20

In [87]:
%%time
df = pd.DataFrame()
pages = min(int(lastpage), pages_to_fetch)
for page in range(1, pages + 1):
    pg_url = '{}&page={}'.format(url, page)
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index = True)
    tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
    print('[{}] {} ({}) : {:04d}/{:04d} pages are downloading...'.
        format(tmnow, company, code, page, pages), end="\r")
df = df.rename(columns={'날짜':'date','종가':'close','전일비':'diff'
    ,'시가':'open','고가':'high','저가':'low','거래량':'volume'})
df['date'] = df['date'].replace('.', '-')
df = df.dropna()
df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close',
    'diff', 'open', 'high', 'low', 'volume']].astype(int)
df = df[['date', 'open', 'high', 'low', 'close', 'diff', 'volume']]

Wall time: 1.5 s8] 동화약품 (000020) : 0020/0020 pages are downloading...
